# CT-LTI: Figure 4: Generate Evaluation Data
For Figure 4 some data need to be generate after training to calculate metrics over epochs.
Please run this script to recalculate the evaluation if you do not use the provided results.

Please make sure that the required data folder is available at the paths used by the script.
You may generate the required data by running the python script
```nodec_experiments/ct_lti/gen_parameters.py```.

Please also make sure that a training proceedure has produced results in the corresponding paths used below.
Running ```nodec_experiments/ct_lti/single_sample/train.ipynb``` with default paths is expected to generate at the requiered location.

As neural network intialization is stochastic, please make sure that appropriate seeds are used or expect some variance to paper results.

## Imports

In [ ]:
# %load_ext autoreload
# %autoreload 2

In [ ]:
import os
os.sys.path.append('../../../')

import torch
import numpy as np

import pandas as pd

from tqdm.cli import tqdm

import plotly
import plotly.express as px

from nnc.controllers.baselines.ct_lti.dynamics import ContinuousTimeInvariantDynamics
from nnc.controllers.baselines.ct_lti.optimal_controllers import ControllabiltyGrammianController

from nnc.helpers.torch_utils.graphs import adjacency_tensor, drivers_to_tensor
from nnc.helpers.graph_helper import load_graph
from nnc.helpers.torch_utils.evaluators import FixedInteractionEvaluator
from nnc.helpers.torch_utils.losses import FinalStepMSE
from nnc.helpers.torch_utils.trainers import NODECTrainer
from nnc.helpers.torch_utils.file_helpers import read_tensor_from_collection

from nnc.controllers.neural_network.nnc_controllers import NNCDynamics
from nnc.helpers.torch_utils.nn_architectures.fully_connected import StackedDenseTimeControl
from nnc.helpers.plot_helper import base_layout, sci_notation, ColorRegistry

## Load sample and parameters
Here we load the sample that we trained NODEC on as well as the parameters for the dynamics.

In [ ]:
device = 'cpu'
graph='lattice'

# load graph data
experiment_data_folder = '../../../../data/parameters/ct_lti/'

# training results are expected to already have been produced and moved to the data folder.
training_results_data_folder = '../../../../data/results/ct_lti/single_sample/'


results_data_folder = '../../../../results/ct_lti/single_sample/'

os.makedirs(results_data_folder, exist_ok=True)

graph_folder = experiment_data_folder+graph+'/'
adj_matrix = torch.load(graph_folder+'adjacency.pt').to(dtype=torch.float, device=device)
n_nodes = adj_matrix.shape[0]
drivers = torch.load(graph_folder + 'drivers.pt')
n_drivers = len(drivers)
pos = pd.read_csv(graph_folder + 'pos.csv').set_index('index').values
driver_matrix = drivers_to_tensor(n_nodes, drivers).to(device)


target_states = torch.load(graph_folder+'target_states.pt').to(device)
initial_states = torch.load(experiment_data_folder+'init_states.pt').to(device)

current_sample_id = 24

x0 = initial_states[current_sample_id].unsqueeze(0)
xstar = target_states[current_sample_id].unsqueeze(0)

# total time for control

total_time=0.5

# select dynamics type and initial-target states

dyn = ContinuousTimeInvariantDynamics(adj_matrix, driver_matrix)

# Below is a helper function that loads parameters from a specific epoch and uses them to evaluate.
def check_for_params(params, n_interactions, logdir=None, epoch=0):
    nn = StackedDenseTimeControl(n_nodes, 
                                 n_drivers, 
                                 n_hidden=0, 
                                 hidden_size=15,
                                 activation=torch.nn.functional.elu,
                                 use_bias=True
                                ).to(x0.device)

    nndyn = NNCDynamics(dyn, nn).to(x0.device)
    nndyn.nnc.load_state_dict(params)


    loss_fn = FinalStepMSE(xstar, total_time=total_time)

    nn_evaluator = FixedInteractionEvaluator(
        'early_eval_nn_sample_ninter_' + str(n_interactions),
        log_dir=logdir,
        n_interactions=n_interactions,
        loss_fn=loss_fn,
        ode_solver=None,
        ode_solver_kwargs={'method' : 'dopri5'},
        preserve_intermediate_states=False,
        preserve_intermediate_controls=False,
        preserve_intermediate_times=False,
        preserve_intermediate_energies=False,
        preserve_intermediate_losses=False,
        preserve_params=False,
    )
    nn_res = nn_evaluator.evaluate(dyn, nndyn.nnc, x0, total_time, epoch=epoch)
    return nn_evaluator, nn_res

all_epochs = pd.read_csv(training_results_data_folder + 'nn_sample_train/epoch_metadata.csv')['epoch']


##  Generating Data
For this figure we first need to load all stored parameters per epoch and evaluate them for all 3 different interaction intervals $10^{-2}, 10^{-3}, 10^{-4}$. Since this is a costly operation, we can also choose to reload an existing file if there is one. 

In [ ]:
# In this case we calculate the loss and energy values per epoch by using the evaluator. 
# This is expected to be a slow calculation.
loss_50  = [] 
loss_500 =[]
loss_5000 = []

for epoch in tqdm(all_epochs):
    params = read_tensor_from_collection(training_results_data_folder + 'nn_sample_train/' + 'epochs', 'nodec_params/ep_'+str(epoch)+'.pt')
    loss_50.append(check_for_params(params, 50)[1]['final_loss'])
    loss_500.append(check_for_params(params, 500)[1]['final_loss'])
    loss_5000.append(check_for_params(params, 5000)[1]['final_loss'])

losses_df = pd.DataFrame({
    50 : torch.stack(loss_50).cpu().detach().numpy(),
    500 :  torch.stack(loss_500).cpu().detach().numpy(),
    5000 :  torch.stack(loss_5000).cpu().detach().numpy()    
}, index = pd.Series(all_epochs, name='Epoch'))

losses_df.to_csv(results_data_folder + 'nn_sample_train/losses_interactions_training.csv')